<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/DecisionIntelligenceSemanticKernelLogo.png" width="60px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Premortem (What-If) Decision Analysis  </span>
</div>

> 📜 **_"The pre-mortem method has a number of benefits besides identifying trouble spots in a plan – reducing overconfidence and promoting discoveries. The method can strengthen members’ mental models… And it can help create a culture of candor and trust."_**
>
> -- <cite>Gary Klein (Psychologist, creator of the Pre-mortem method)</cite>  

Decision Intelligence applied in this module:  
* Using Premortem Decision Analysis to improve decision quality & outcomes  
* Using a Premortem decision output for improving of Gathering Intelligence 
* Attain different perspectives using mutliple decision frames  
* Apply a Decision Framework to Premortem Decision Analysis 


### What is a Premortem (What-If Analysis)  

You have probably heard of a **postmortem (post-mortem)**, which is a look back at an actual failure (or success) to understand why it happened and learn from it. In the context of decision making, after the decision outcome is clear you can analyze why the result occured. For example, a hedge fund may host a postmortem meeting after analyzing an unsuccesful move on a tech stock that didn't materialize the return on investment as was planned. 

In a **premortem (pre-mortem)** the outcome is "imagined" and then it is figured out why that potential "imagined" outcome happened. In a premortem, a planning exercise is held where a team imagines that a project or decision has already succeeded or failed. The team then works backward, brainstorming what might have caused things to go wrong. This approach allows everyone to spot potential risks and address them before starting the project. Therefore, it’s a bit like predicting the future and using that prediction to make the forecast stronger. 

### Example Decision Scenario - Purchasing a Car   

Imagine you're about to buy a new car. Let's play a game of "What-If" that the purchase decision results in either two outcomes a: **disaster** or a **successful** outcome. 

#### Premortem Outcome - Disaster  

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-DecisionPremortem.png">

In a **premortem (pre-mortem)**, you pretend the car purchase turned out to be a disaster. Now, you ask yourself:  
**What went wrong with the decision to buy a new car?**

Here are some potential reasons the car purchase had a poor outcome:  

* **Financial issues:** You realize later you can’t comfortably afford the monthly payments or maintenance costs.  
* **Unexpected repairs:** The car has hidden mechanical problems, leading to costly repairs.  
* **Insurance costs:** Insurance is much higher than expected, impacting your budget.
* **Poor fuel efficiency:** The car uses more gas than you thought, making it expensive to drive.
Resale value drop: The car’s value plummets quickly, leaving you with less if you want to sell or trade it.  

#### Premortem Outcome - Success

In a premortem for a successful car purchase, imagine you bought the car, and it turned out to be a succssful decision. Now, you ask yourself:  
**What went right with the decision to buy a new car?**  

Here are some potential reasons the car purchase had a good outcome: 

* **Affordable and realistic budget:** You set a budget that allowed for comfortable monthly payments and covered insurance, fuel, and maintenance without straining your finances.  
* **Thorough inspection:** You got a detailed inspection before buying, which confirmed the car was in excellent condition and minimized the risk of future repairs.
* **Reliable model choice:** You researched models with a reputation for reliability, so you experience fewer repairs and smooth performance.
* **Fuel efficiency matches needs:** You chose a car that meets your fuel efficiency needs, saving money on gas and fitting well with your daily driving habits.   
* **High resale value:** You picked a make and model that holds its value well, giving you confidence that it’s a good long-term investment.  

#### Premortem (What-If) Decision Analysis 

Imagining successful outcomes helps you focus on actions like sticking to a car budget, getting a full inspection, and choosing a reliable model. You will be more likely to make a purchase you’ll be happy with. By plyaing a "game" of What-If with decision criteria in advance, you could adapt your strategy. For example, you could consider to thoroughly check maintenance records, buy additional warranty, test drive with similar terrain in your area, or reassess your budget to avoid these potential problems.

How can Generative AI help? Using Generative AI, the LLM models can be used to "imagine" premortem outcomes and  brainstorm potential factors why that certain outcome occured. Generative AI agents can act as your "premortem" team. This can lead to additional questions, gathering of intelligence, checklists, negotiation leverage etc. you didn't consider initially. 

---
### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

In [16]:
#r "nuget: Microsoft.Extensions.Configuration, 9.0.6"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.6"
#r "nuget: Microsoft.SemanticKernel, 1.60.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.60.0-preview"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info using o1 series of models
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.6 Microsoft.Extensions.Configuration.Json, 9.0.6 Microsoft.SemanticKernel, 1.60.0 Microsoft.SemanticKernel.Plugins.Core, 1.60.0-preview

In [17]:
using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Using Azure OpenAI Service


----
### Step 2 - Premortem Risk Analysis with AI Reasoning Models

> 📜 **_"Success is a lousy teacher. It seduces smart people into thinking they can’t lose."_**
>
> -- <cite>Bill Gates (Co-Founder of Microsoft, Philantropist)</cite>  

In the decision scenario below, an AI model will perform a premortem (What-If) analysis on the decision whether to purchase or lease a car. The AI model will provide risks and items to consider for each of the decision options: purchasing a car or leasing a car. 

In [18]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. Provide structured, logical, and comprehensive advice.

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";
var decisionPreMortemAnalysisPrompt = """
Perform an advanced premortem (What-If) analysis on the following decision scenario:
You are deciding to purchase a new car versus leasing a car.
""";

// Create a new chat history with pre-mortem analysis prompt
var chatHistoryPreMortemAnalysis = new ChatHistory();
chatHistoryPreMortemAnalysis.AddSystemMessage(systemPromptForDecisions);
chatHistoryPreMortemAnalysis.AddUserMessage(decisionPreMortemAnalysisPrompt);

// Create a new OpenAI prompt execution settings object
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

// Include both the chat history and the prompt execution settings in the call to GetStreamingChatMessageContentsAsync
var decisionResponseChainOfThought = await chatCompletionService.GetChatMessageContentAsync(chatHistoryPreMortemAnalysis, openAIPromptExecutionSettings);
decisionResponseChainOfThought.Content.DisplayAs("text/markdown");

// Add the AI response to the chat history
chatHistoryPreMortemAnalysis.AddAssistantMessage(decisionResponseChainOfThought.Content);


| Step/Aspect                          | Purchase a New Car                                                                                     | Lease a Car                                                                                           |
|---------------------------------------|-------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------|
| **Objective/Goal**                    | Own a reliable vehicle with long-term value and flexibility.                                           | Access a new car with lower upfront costs and regular upgrades.                                       |
| **Key Assumptions**                   | Stable income, plan to keep car for several years, can handle maintenance/repairs.                     | Stable income, prefer new cars every few years, drive within mileage limits, minimal maintenance.      |
| **Success Criteria**                  | Cost-effective over 5+ years, high resale value, meets lifestyle needs.                                | Lower monthly payments, minimal hassle, access to latest features, predictable costs.                  |
| **What-If: Unexpected Job Loss**      | May struggle with loan payments, but can sell the car to recover equity or reduce debt.                | Lease payments still required; early termination fees can be high; no asset to sell.                   |
| **What-If: Major Life Change (move, family growth)** | Car ownership allows flexibility to sell or trade for a different vehicle as needs change.              | Lease may restrict ability to change vehicles; early termination penalties; possible transfer options. |
| **What-If: High Annual Mileage**      | No penalty for high mileage, but higher depreciation and maintenance costs.                             | Significant extra charges for exceeding mileage limits.                                                |
| **What-If: Vehicle Depreciation Accelerates** | Lower resale value, higher total cost of ownership, but still own the asset.                            | Not your concern; residual value risk is on the leasing company.                                       |
| **What-If: Unexpected Repairs**       | Responsible for all repairs after warranty expires; can be costly.                                      | Most repairs covered under warranty due to short lease term; less risk of major repair costs.          |
| **What-If: Desire to Customize**      | Full freedom to modify or personalize the car.                                                         | Customization usually not allowed; must return car in original condition.                              |
| **What-If: Interest Rate Increases**  | Higher financing costs if not locked in; may affect affordability.                                     | Lease rates may also rise, but typically less sensitive than loans.                                    |
| **What-If: Market Value Drops Sharply** | Owner absorbs loss in value; higher depreciation.                                                       | Leasing company absorbs risk; your payments remain the same.                                           |
| **What-If: Want to Exit Early**       | Can sell or trade in the car, possibly at a loss; more flexibility.                                    | Early termination fees can be substantial; less flexibility.                                           |
| **What-If: Financial Windfall**       | Can pay off car early or upgrade; asset can be sold for cash.                                          | May not be able to buy out lease easily; less financial flexibility.                                   |
| **What-If: Environmental Concerns**   | Can choose fuel-efficient or electric vehicle and keep it long-term.                                   | Can upgrade to latest eco-friendly models more frequently.                                             |
| **Long-Term Cost Uncertainty**        | Maintenance, repairs, and depreciation can be unpredictable.                                           | Predictable monthly payments, but possible end-of-lease charges.                                       |
| **End-of-Term Options**               | Keep, sell, or trade in the car; full control.                                                        | Return car, buy it at residual value, or lease a new one.                                              |
| **Potential Hidden Costs**            | Taxes, insurance, maintenance, unexpected repairs.                                                     | Excess mileage, wear-and-tear charges, disposition fees, insurance.                                    |
| **Emotional/Intangible Factors**      | Pride of ownership, emotional attachment, stability.                                                   | Enjoyment of driving new cars, less attachment, less hassle.                                           |
| **Summary of Main Risks**             | Depreciation, repair costs, inflexibility if needs change, financial risk if unable to pay.            | Mileage overages, early termination fees, lack of equity, limited customization.                       |
| **Summary of Main Opportunities**     | Build equity, long-term savings, full control, flexibility.                                            | Lower upfront costs, regular upgrades, lower maintenance risk, predictable expenses.                   |

The large reasoning model provided a comprehensive set of criteria, risks, mitigation strategy and approaches for the two decisions. This is a great option, but it might be more ideal to provide this in an easy to consume table (checklist) for the user that is easy to compare and contrast. This What-If analysis is easy to consume on a mobile device or even print out and annotate. 

This Markdown table can be further simplified by highlighting which risk factor is more favorable when comparing a car purchase versus a car lease. This can make the decision process much easier to the user, as at a glance they can see overwhelming evidence that favors once decision option over the other.  

In [19]:
var simplifyRiskAnalysisPrompt = """
You are provided a detailed markdown table in the above chat history that 
compares Purchase and Lease options across various risk factors and uncertainties. 
Your task is to convert this table into a simplified markdown table that uses check marks (☑️) 
to indicate which option is less risky or more ideal for each risk factor. 
The output table should have three columns: "Risk Factor / Uncertainty / Potential Outcome", "Purchase", and "Lease". 
For each risk factor, decide based on the detailed descriptions which option (Purchase or Lease) 
is preferable (i.e., less risky or more advantageous) and place a check mark (☑️) in that column. 
If an option is not the ideal choice for a given risk factor, leave its cell blank. 
Make sure the simplified table closely reflects the key insights from the detailed table. 

Provide a small summary below the table of the key insights 
from the detailed table in a single paragraph before the simplified table.
""";

chatHistoryPreMortemAnalysis.AddUserMessage(simplifyRiskAnalysisPrompt);

var preMortemAnalysisSimplifiedChecklistResponse = await chatCompletionService.GetChatMessageContentAsync(chatHistoryPreMortemAnalysis, openAIPromptExecutionSettings);
preMortemAnalysisSimplifiedChecklistResponse.Content.DisplayAs("text/markdown");


The detailed analysis shows that purchasing a car is generally more advantageous for those seeking long-term flexibility, equity, and control, especially if they expect high mileage, want to customize, or may need to change vehicles due to life changes. Leasing, on the other hand, is preferable for those prioritizing lower upfront costs, predictable payments, minimal maintenance risk, and frequent access to newer vehicles. Leasing also protects against depreciation and unexpected repair costs during the lease term, but can be restrictive if your needs change or if you exceed mileage limits.

| Risk Factor / Uncertainty / Potential Outcome      | Purchase | Lease |
|---------------------------------------------------|----------|-------|
| Unexpected Job Loss                               | ☑️       |       |
| Major Life Change (move, family growth)           | ☑️       |       |
| High Annual Mileage                               | ☑️       |       |
| Vehicle Depreciation Accelerates                  |          | ☑️    |
| Unexpected Repairs                                |          | ☑️    |
| Desire to Customize                               | ☑️       |       |
| Interest Rate Increases                           |          | ☑️    |
| Market Value Drops Sharply                        |          | ☑️    |
| Want to Exit Early                               | ☑️       |       |
| Financial Windfall                                | ☑️       |       |
| Environmental Concerns                            |          | ☑️    |
| Long-Term Cost Uncertainty                        |          | ☑️    |
| End-of-Term Options                               | ☑️       |       |
| Potential Hidden Costs                            |          | ☑️    |
| Emotional/Intangible Factors                      | ☑️       |       |

### Step 3 - Premortem Analysis for Radio Telescope Operations 

Exploring space with telescopes is a time consuming investment. Current telescope designs are limited to exploring small areas of space in a certain time frame. Without going into details, the process is much more complex than snapping a picture with a mobile device. Telescopes can be pointed at a specific space area for days, weeks or even more than months! If a telescope is exploring a certain area of space it isn't collecting data on other areas. Scientists and researchers compete for valuable access to telescope resources by providing observatories what space area they want to explore, any cellestial contraints, timeframes, calibration costs, data collection transfer etc. This presents a very interesting Decision Intelligence constraints problem that observatories must manage. 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-RadioTelescope.png">

#### Decision Scenario - Premortem for Radio Telescope Operations 

> 📜 "We choose to go to the Moon in this decade and do the other things, not because they are easy, but because they are hard." 
>
> -- <cite>John F. Kennedy (35th President of the United States)</cite>  

A newly commissioned radio telescope stands on the verge of unveiling the proof of alien life, but it is still far from fully calibrated. Despite limited data processing capacity and a software suite that remains untested, an unexpected cosmic event has just begun, hinting at signals that could reveal alien life. Faced with the risk of hardware damage or data corruption from running the telescope prematurely, scientists debate whether to switch it on now or wait until everything is fully operational—knowing they could lose a once-in-a-generation opportunity. Determined not to squander this brief cosmic window, the team conducts a premortem analysis to predict what might go wrong. Armed with these insights, they can better prepare for potential pitfalls and strengthen their chances of catching historic signals from across the galaxy.

Let's assume the scientists decide to rush the radio telescope to operate early to find alien life. Furthermore, assume a that this decision has led to a disaster with hardware failure to the telescope. Even though the (What-If) decision was made to rush the radio telescope operationally, formal scientific telescope operations have been followed. The goal of this premortem (What-If analysis) is to illuminate any risk mitigation strategies and craft a risk mitigation approach that may have been uncovered based on the premortem. 

Techinques like this a very powerful in a variety of business scenarios, where existing operational frameworks exist but simulating an outcome can uncover gaps, emerging risks and other opportunities to optimize. 

In [20]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForPremortemRadioTelescope = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. Provide structured, logical, and comprehensive advice.

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";
var decisionAnalysisPromptForPremortemRadioTelescope = """
You are a project manager at a new radio telescope facility looking for alien life.  
It is still far from fully calibrated. Despite limited data processing capacity and a software suite that remains untested, 
an unexpected cosmic event has just begun, hinting at signals that could reveal alien life. 
Faced with the risk of hardware damage or data corruption from running the telescope prematurely, 
scientists debate whether to switch it on now or wait until everything is fully operational—knowing 
they could lose a once-in-a-generation opportunity. 

Imagine it is six months from now and the project to detect potential alien signals from the sudden cosmic event has failed. 
The radio telescope was likely activated prematurely, software remained untested, and the team rushed to capture transmissions. 
Despite the urgency, the outcome was disastrous: hardware suffered damage under the increased load, 
data came out corrupted, or no meaningful signals were confirmed. 

Now, step into that moment of failure—what specific issues, oversights, or cascading errors caused this grim scenario?

1) Envision every possible way the radio telescope's early deployment could have led to compromised or unusable data.
2) Identify how incomplete software testing and rushed calibration might have 
contributed to errors in processing or storing signals. 
3) Consider any overlooked safety protocols or hardware precautions that might have led to damage or malfunctions.
4) Examine the roles and responsibilities of the team and think about communication breakdowns, 
rushed decisions, and missed signals or deadlines.
5) Assess how time pressure from the rare cosmic event might have amplified smaller mistakes and 
turned them into systemic failures.

Use the outcomes of this “what if” vision to create a detailed list of preventive measures. 
Then, propose fail-safes, additional resources, or procedural changes to ensure that, 
should you choose to activate the telescope early, you can still capture the event safely and 
effectively—maximizing the chance to detect historic signals without jeopardizing the entire operation.
""";

// Create a new chat history with pre-mortem analysis prompt
var chatHistoryPreMortemTelescopeAnalysis = new ChatHistory();
chatHistoryPreMortemTelescopeAnalysis.AddSystemMessage(systemPromptForPremortemRadioTelescope);
chatHistoryPreMortemTelescopeAnalysis.AddUserMessage(decisionAnalysisPromptForPremortemRadioTelescope);

// Create a new OpenAI prompt execution settings object
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 8000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var preMortemTelescopeAnalysisResponse = await chatCompletionService.GetChatMessageContentAsync(chatHistoryPreMortemTelescopeAnalysis, openAIPromptExecutionSettings);
preMortemTelescopeAnalysisResponse.Content.DisplayAs("text/markdown");

| Category | Failure Scenario (Six Months Later) | Root Causes & Contributing Factors | Preventive Measures | Fail-safes, Resources, & Procedural Changes |
|----------|-------------------------------------|------------------------------------|---------------------|--------------------------------------------|
| **Data Integrity & Usability** | Data from the cosmic event is corrupted, incomplete, or unusable for analysis. | - Uncalibrated sensors led to inaccurate readings.<br>- Data pipelines not validated, causing loss or mislabeling of signals.<br>- Storage systems not stress-tested, resulting in dropped packets or overwrites.<br>- Lack of redundancy in data capture.<br>- No real-time data quality monitoring. | - Conduct partial calibration on critical subsystems.<br>- Implement minimum viable data validation checks.<br>- Prioritize redundancy in data storage.<br>- Schedule parallel dry runs using simulated data.<br>- Develop a data integrity checklist for early activation. | - Set up real-time data integrity dashboards.<br>- Use cloud-based backup for raw data.<br>- Assign a rapid-response data triage team.<br>- Predefine a protocol for immediate data review and triage. |
| **Software Functionality** | Software crashes, fails to process signals, or introduces errors during analysis and storage. | - Core modules untested under real-world loads.<br>- Incomplete error handling.<br>- Incompatibility between software and hardware.<br>- No rollback plan for failed updates.<br>- Lack of automated monitoring or alerting. | - Prioritize end-to-end testing of critical software paths.<br>- Run stress tests with simulated high-load events.<br>- Develop a rollback plan for software failures.<br>- Implement automated error logging and alerts.<br>- Document known bugs and workarounds. | - Deploy a sandbox environment for live patching.<br>- Assign dedicated software engineers on-call during event.<br>- Use containerization to isolate unstable modules.<br>- Establish clear escalation paths for software issues. |
| **Hardware Safety & Reliability** | Telescope components overheat, fail, or suffer irreversible damage. | - Safety interlocks bypassed or not fully configured.<br>- Cooling systems not tested under sustained load.<br>- Power surges or overloads not anticipated.<br>- Lack of real-time hardware health monitoring.<br>- No emergency shutdown protocol. | - Verify all critical safety interlocks.<br>- Conduct limited-duration load tests.<br>- Pre-inspect cooling and power systems.<br>- Install hardware health sensors.<br>- Develop and rehearse emergency shutdown procedures. | - Set up automated shutdown triggers for abnormal readings.<br>- Keep spare parts and repair kits on-site.<br>- Assign a hardware safety officer for the event.<br>- Maintain a hotline to hardware vendors for rapid support. |
| **Team Coordination & Communication** | Confusion, duplicated efforts, or missed critical tasks; slow response to emerging issues. | - Roles and responsibilities unclear during crisis.<br>- Incomplete shift handovers.<br>- Poor documentation of decisions.<br>- Critical information not disseminated in real time.<br>- No central incident log or command structure. | - Define and communicate clear roles for all team members.<br>- Establish a command center for the event.<br>- Use checklists for task tracking.<br>- Schedule regular briefings and debriefings.<br>- Maintain a shared incident log accessible to all. | - Implement a “single point of truth” dashboard.<br>- Assign incident commanders for each subsystem.<br>- Use radio or instant messaging for rapid updates.<br>- Conduct pre-event drills to test communication flow. |
| **Decision-Making Under Pressure** | Rushed or poorly considered decisions amplify minor errors into systemic failures. | - Inadequate risk assessment.<br>- Pressure to act overrides caution.<br>- No structured process for go/no-go decisions.<br>- Insufficient time allocated for contingency planning.<br>- Overreliance on optimistic assumptions. | - Develop a decision matrix for early activation scenarios.<br>- Require sign-off from technical leads for critical steps.<br>- Schedule “red team” reviews to challenge assumptions.<br>- Allocate time for scenario planning and risk assessment.<br>- Document all decisions and rationales. | - Establish a “pause and review” protocol for high-risk actions.<br>- Empower any team member to call for a halt if safety/data integrity is at risk.<br>- Bring in external advisors for independent review.<br>- Use decision logs for post-event analysis and learning. |

